In [1]:
import time
import math as m
import pandas as pd
import requests
from nba_api.stats.endpoints import leaguegamefinder, boxscoretraditionalv2
from nba_api.stats.library.parameters import SeasonType, SeasonID, LeagueID

In [8]:
# Read current CSV and get max date pulled
path = "C:/Users/zacha/OneDrive/Desktop/sports_data_storage/sports/nba/player_props/tableau_data_ao_2022-02-02.csv"


In [9]:
df = pd.read_csv(path)

In [10]:
dates = df['GAME_DATE'].unique()

In [12]:
max_old_date = max(dates)
season_yr = '2021-22'
max_old_date

'2022-02-01'

In [13]:
gamefinder = leaguegamefinder.LeagueGameFinder(season_type_nullable=SeasonType.regular,season_nullable=season_yr,
                                               league_id_nullable=LeagueID.nba)

In [14]:
games = gamefinder.get_data_frames()[0]

In [15]:
new_games = games[games['GAME_DATE']>max_old_date]

In [16]:
from datetime import date
today = date.today()
print("Today's date:", today)

Today's date: 2022-02-17


In [17]:
games.to_csv(f"C:/Users/zacha/OneDrive/Desktop/sports_data_storage/sports/nba/team_box_scores_ao_{today}.csv",index = False)

In [20]:
all_new_games = list(new_games['GAME_ID'].unique())
len(all_new_games)

113

In [21]:
def get_box_score_df(g_id):
    bs = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id = g_id).get_data_frames()[0]
    return bs

In [22]:
from time import sleep
import sys

all_results = []

N = len(all_new_games)

print(f"Reading {N} games from NBA_API")

n = 0
i = 0
partition = int(N/50)
for g_id in all_new_games:
    if (n % partition == 0) or (n==0):
        sys.stdout.write('\r')
        # the exact output you're looking for:
        sys.stdout.write("[%-50s] %d%%" % ('='*i, 2*i))
        sys.stdout.flush()
        i += 1
    all_results.append(get_box_score_df(g_id))
    sleep(1)
    n += 1

Reading 113 games from NBA_API
[========================================================] 112%

In [23]:
new_player_df = pd.concat(all_results)

In [24]:
new_player_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS'],
      dtype='object')

In [28]:
new_player_df.to_csv("C:/Users/zacha/OneDrive/Desktop/sports_data_storage/sports/nba/player_box_scores_ao_2022-02-17.csv",
                    index = False)

In [27]:
p_2 = "C:/Users/zacha/OneDrive/Desktop/sports_data_storage/sports/nba/player_box_scores_ao_2022-02-02.csv"
old_player_df = pd.read_csv(p_2)
old_player_df

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/zacha/OneDrive/Desktop/sports_data_storage/sports/nba/player_box_scores_ao_2022-02-02.csv'

In [44]:
frames = [old_player_df,new_player_df]
result = pd.concat(frames)

In [46]:
result.to_csv(f"C:/Users/zacha/OneDrive/Desktop/sports_data_storage/sports/nba/player_box_scores_ao_{today}.csv",index = False)